In [ ]:

from requests import session
import matplotlib.pyplot as plt
import pandas as pd
import json
import copy
import numpy as np, scipy.stats as st
import json
from tqdm import tqdm
import pickle
import logging
import  argparse
import math
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from itertools import chain
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
logging.basicConfig(level=logging.INFO)
from sklearn import preprocessing
import matplotlib
from re import S
from scipy.stats.stats import pearsonr
from sklearn.calibration import calibration_curve
import matplotlib
from sklearn.metrics import roc_auc_score
import random
import sys
import os
# sys import ..
sys.path.append("../")
from data_split import *
from metrics_get import *
from collections import Counter



In [ ]:
path = '../data/featureframe_user_study.pkl'
data = pickle.load(open(path, 'rb'))
splitbyusers = True
output_path = '.'
test_percentage = 0.001
val_percentage = 0.001
REMOVE_S_AND_R = False # remove shown and replay
feature_dict = {'Measurements: compCharLen, confidence, documentLength, numLines, numTokens, promptCharLen, promptEndPos, quantile': 0,
'edit percentage': 1, 'time_in_state': 2, 'session_features':3, 'suggestion_label':4, 'prompt_label':5,
'suggestion_embedding':6, 'prompt_embedding':7, 'suggestion_text_features':8, 'prompt_text_features':9, 'user_id':10, 'statename':11, 'labeled_state': 12}
# split into train and test
SEQUENCE_MODE = False # keep session as a sequence or split it into events
SPLIT_BY_USER = bool(splitbyusers) # otherwise split by session uniformly
ADD_PREVIOUS_STATES = True
PREDICT_ACTION = True # Otherwise predict time in state
NORMALIZE_DATA = False # normalize data

previous_states_to_keep = 5
if not PREDICT_ACTION and SPLIT_BY_USER:
    raise ValueError('Cannot predict time and split by user')




In [ ]:
# all Counter({0: 276, 2: 235, 4: 172, 9: 121, 3: 68, 11: 63, 7: 40, 6: 39, 8: 39, 5: 36, 10: 5, 12: 1, 1: 1})
#new_label_map = {0:0, 2:1,4:2,3:4,1:4,5:4,6:4,7:4,8:4,9:4,10:4,11:4,12:4}
new_label_map = {0:0, 2:4,4:4,3:4,1:4,5:4,6:4,7:4,8:4,9:4,10:4,11:4,12:4}
new_label_map = {0:0, 2:2,4:4,3:3,1:1,5:5,6:6,7:7,8:8,9:9,10:10,11:11,12:12}
def transform_label(y):
    return np.array([new_label_map[y_] for y_ in y])

def normalized_counter(elements):
    counter = Counter(elements)

    # Calculate the length of the list
    list_length = len(elements)

    # Normalize frequencies by dividing by the list length
    normalized_frequencies = {element: frequency / list_length for element, frequency in counter.items()}

    return normalized_frequencies

In [ ]:
MAX_TRIALS = 21 # number of programmers
so_far_count = 0
accs = []
baseline_accs = []
aucs = []
baseline_all0 = []
baseline_all4 = []

test_set_seenns = []
while so_far_count < MAX_TRIALS:
    random_number = np.random.randint(0, 100000)
    # without latent state

    features_to_keep = np.array([0,2,3,4,5,8,9,10])
    label_index = np.array([11])
    df_observations_features, df_observations_labels = get_features_labels_user_study(path, features_to_keep, label_index, REMOVE_S_AND_R, False)

    X_train, X_test, X_val, y_train, y_test, y_val = process_data_user_study(df_observations_features, df_observations_labels,
    REMOVE_S_AND_R, SEQUENCE_MODE, SPLIT_BY_USER, ADD_PREVIOUS_STATES, PREDICT_ACTION, NORMALIZE_DATA,
    test_percentage, val_percentage, previous_states_to_keep, random_number)

    label_index = np.array([12])
    df_observations_features, df_observations_labels = get_features_labels_user_study(path, features_to_keep, label_index, REMOVE_S_AND_R, True)

    _, _, _, y_train, y_test, y_val = process_data_user_study(df_observations_features, df_observations_labels,
    REMOVE_S_AND_R, SEQUENCE_MODE, SPLIT_BY_USER, ADD_PREVIOUS_STATES, PREDICT_ACTION, NORMALIZE_DATA,
    test_percentage, val_percentage, previous_states_to_keep, random_number)
    
    y_test = transform_label(y_test)
    y_train = transform_label(y_train)
    y_val = transform_label(y_val)
    # check if X_test array is already been seen
    for seen in test_set_seenns:
        if np.array_equal(X_test, seen):
            continue
    test_set_seenns.append(X_test)
    so_far_count += 1
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba = y_pred_proba[:,1]
    #auc = roc_auc_score(y_test, y_pred_proba)
    #aucs_without.append(auc)
    accs.append(accuracy)
    # Counter y_test with dividing by len
    normalized_frequencies = normalized_counter(y_test)
    print(f'Counter y_test: {normalized_frequencies}')
    max_element = max(normalized_frequencies, key=normalized_frequencies.get)
    max_normalized_frequency = normalized_frequencies[max_element]
    baseline_accs.append(max_normalized_frequency)
    baseline_all0.append(normalized_frequencies[0])
    baseline_all4.append(normalized_frequencies[4])
    print(f'baseline_accs: {baseline_accs}')
    print(f'accs_without: {accs}')
    print(f'baseline_all0: {baseline_all0}')
    print(f'baseline_all4: {baseline_all4}')
    #auc = roc_auc_score(y_test, y_pred_proba)
    #aucs.append(auc)
    #print(f'aucs: {aucs}')

In [ ]:
# print mean and standard error
print(f'xgb model: accuracy = {np.mean(accs)} +- {np.std(accs)/np.sqrt(len(accs))}')
print(f'baseline_all0 : accuracy = {np.mean(baseline_all0)} +- {np.std(baseline_all0)/np.sqrt(len(baseline_all0))}')
print(f'baseline_accs : accuracy = {np.mean(baseline_accs)} +- {np.std(baseline_accs)/np.sqrt(len(baseline_accs))}')
print(f'baseline_all4 : accuracy = {np.mean(baseline_all4)} +- {np.std(baseline_all4)/np.sqrt(len(baseline_all4))}')
print(f'xgb model: auc = {np.mean(aucs)} +- {np.std(aucs)/np.sqrt(len(aucs))}')